In [1]:
#!pip install mlflow

In [2]:
import os
import warnings
import sys

In [3]:
import pandas as pd
import numpy as np

In [4]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

In [5]:
from urllib.parse import urlparse

In [6]:
import mlflow
import mlflow.sklearn

In [7]:
import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

In [8]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [9]:
def download_dataset():
    import urllib.request
    import zipfile
    
    url = "https://archive.ics.uci.edu/static/public/186/wine+quality.zip"
    filename = './data/wine.zip'
    urllib.request.urlretrieve(url, filename)
    
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall('./data/')


In [34]:
if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    download_dataset()

    try:
        data = pd.read_csv("./data/winequality-red.csv", sep=";")
    except Exception as e:
        logger.exception(str(e))

    train, test = train_test_split(data)
    train_X = train.drop(["quality"], axis=1)
    train_y = train[["quality"]]

    test_X = test.drop(["quality"], axis=1)
    test_y = test[["quality"]]

    alpha = 0.5
    l1_ratio = 0.5

    with mlflow.start_run():
        lr = ElasticNet(alpha = alpha, l1_ratio = l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        
        predicted_qualities = lr.predict(test_X)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("RMSE {}".format(rmse))
        print("MAE {}".format(mae))
        print("R2 {}".format(r2))
        print("alpha {}".format(alpha))
        print("l1_ratio {}".format(l1_ratio))
        print(mlflow.get_tracking_uri())
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        print(tracking_url_type_store)

        if tracking_url_type_store == "file":
            mlflow.sklearn.log_model(lr, "model", registered_model_name="ElasticWineModel")
        else:
            mlflow.sklearn.log_model(lr, "model")
            


RMSE 0.7931640229276851
MAE 0.6271946374319586
R2 0.10862644997792614
alpha 0.5
l1_ratio 0.5
file:///home/ac/projects/mlops_bootcamp/notebooks/mlruns
file


Successfully registered model 'ElasticWineModel'.
Created version '1' of model 'ElasticWineModel'.
